In [ ]:
# 下載所要的套件 [B]
!pip install keras-self-attention
!pip install Attention

In [2]:
# Import necessary libraries [B]
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt   # plotting
import seaborn as sns   # plotting heatmap
import statsmodels.api as sm  # seasonal trend decomposition
import datetime
from statsmodels.graphics import tsaplots   # autocorrelation
from tensorflow.keras.optimizers import Adam #
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, BatchNormalization, TimeDistributed, Flatten, Bidirectional,Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time
from sklearn import preprocessing
from joblib import dump, load
from datetime import datetime, timedelta
# from attention import Attention
from keras_self_attention import SeqSelfAttention
# from tensorflow.keras.utils import plot_model
import os
import csv

%matplotlib inline
# Import necessary libraries [E]

In [ ]:
# 透過dataframe抓取天氣 [B]
def getWeatherCSV(dataframe, datetime):
    demand_dataframe = dataframe.loc[datetime:, :]
    demand_dataframe = demand_dataframe.iloc[0]
    demand_dataframe = pd.DataFrame(demand_dataframe)
    demand_dataframe = demand_dataframe.T
    return demand_dataframe
# 透過dataframe抓取天氣 [E]

In [ ]:
# 體感溫度計算 [B]
def c_feel_like(temperature, wind_speed):
    feels_like = int(13.12+(0.615*float(temperature))-(11.37*(float(wind_speed)*3.6)**0.16)+(0.3965*float(temperature))*((float(wind_speed)*3.6)**0.16))
    return feels_like
# 體感溫度計算 [E]

In [ ]:
# 讀取模型參數，用來重建模型 [B]
def load_weights(self, filepath, by_name=False, exclude=None):
        """Modified version of the corresponding Keras function with
        the addition of multi-GPU support and the ability to exclude
        some layers from loading.
        exclude: list of layer names to exclude
        """
        import h5py
        # Conditional import to support versions of Keras before 2.2
        # TODO: remove in about 6 months (end of 2018)
        try:
            from tensorflow.python.keras.saving import hdf5_format
        except ImportError:
            # Keras before 2.2 used the 'topology' namespace.
            from keras.engine import topology as saving

        if exclude:
            by_name = True

        if h5py is None:
            raise ImportError('`load_weights` requires h5py.')
        f = h5py.File(filepath, mode='r')
        if 'layer_names' not in f.attrs and 'model_weights' in f:
            f = f['model_weights']

        if by_name:
            saving.load_weights_from_hdf5_group_by_name(f, self.layers)
        else:
            hdf5_format.load_weights_from_hdf5_group(f, self.layers)
        if hasattr(f, 'close'):
            f.close()
        elif hasattr(f.file, 'close'):
            f.file.close()
# 讀取模型參數，用來重建模型 [E]

In [ ]:
# 建立模型架構 [B]
def create_model(look_back, feature_len, model_structure):
    if model_structure == 'vanilla':
        # Vanilla LSTM (many to one)
        vanilla_model = Sequential()
        vanilla_model.add(LSTM(50, input_shape=(look_back,feature_len), return_sequences=True)) #X_train.shape[1],X_train.shape[2]
        vanilla_model.add(Attention(50))
        vanilla_model.add(Flatten())
        vanilla_model.add(Dense(1))
        vanilla_model.compile(loss='mse', optimizer='Adam') 
        model = vanilla_model
    if model_structure == 'self-attention':
        vanilla_model = Sequential()
        vanilla_model.add(LSTM(50, input_shape=(look_back,feature_len), return_sequences=True)) #X_train.shape[1],X_train.shape[2]
        vanilla_model.add(SeqSelfAttention(attention_activation='softmax'))
        vanilla_model.add(Flatten())
        vanilla_model.add(Dense(1))
        vanilla_model.compile(loss='mse', optimizer='Adam') 
        model = vanilla_model
    if model_structure == 'no-attention':
        vanilla_model = Sequential()
        vanilla_model.add(LSTM(50, input_shape=(look_back,feature_len)))
        vanilla_model.add(Dense(1, activation='relu'))
        vanilla_model.compile(loss='mse', optimizer='Adam') 
        model = vanilla_model
    return model
# 建立模型架構 [E]

In [ ]:
# 參數轉成監督式參數 [B]
# The function is defined with default parameters so that if you call it with just your data, it will construct a DataFrame with t-1 
# as X and t as y

def series_to_supervised(data, n_in = 1, n_out = 1, dropnan = True):
    n_vars = 1 if type(data) is list else data.shape[1]
    
    df = pd.DataFrame(data)
    df_columns_name = df.columns.values
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (df_columns_name[j], i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (df_columns_name[j])) for j in range(n_vars)]
        else:
            names += [('%s(t+%d)' % (df_columns_name[j], i)) for j in range(n_vars)]
            
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg
# 參數轉成監督式參數 [E]

In [ ]:
# 將預測結果寫入 csv 檔案 [B]
def uploadPredictionCSV(table_name, now_quarter_times, predict_datetime, predict_demands):
    # 上傳預測結果於資料庫 [B]
    i = 0
    delta_times_list = {
      1: 4, # i = 1 意思是相差15分鐘，為最後一次預測
      2: 3, # i = 2 意思是相差30分鐘，為最後二次預測
      3: 2,
      4: 1
    }
    with open("/content/drive/My Drive/ncku_demand/files/"+table_name+".csv", 'a+', newline='') as write_obj:
        for each_predict_datetime in predict_datetime:
            csv_writer = csv.writer(write_obj)
            # [time, prediction_?, demand]
            csv_writer.writerow([each_predict_datetime, delta_times_list[i + 1], predict_demands[i]])
            i += 1
    write_obj.close()
# 將預測結果寫入 csv 檔案 [E]

In [ ]:
''' 需量預測函式，在 codlab 上透過撈取 csv 資訊來做預測 [B]'''
def predict_demand_csv(dataframe, begin_date, database_name, model_structure):
    # 預測間隔設定 [B]
    predict_min = 15
    window_size = 96
    feature_len = 24
    # feature_len = 13
    # 預測間隔設定 [E]
    # 讀取pid路徑 [B]
    zscore_pid_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/pid/zscore-pid.pid'
    encode_pid_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/pid/encode-pid.pid'
    modal_pid_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/pid/modal-pid.pid'
    last_data_datetime_pid_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/pid/last-data-datetime-pid.pid' # 撈的最後一筆資料的時間
    # 讀取pid路徑 [E]
    # 讀取檔案最後日期 [B]
    with open(zscore_pid_path) as f:
        zscore_file_date = f.read()
        f.close()
    with open(encode_pid_path) as f:
        encode_file_date = f.read()
        f.close()
    with open(modal_pid_path) as f:
        modal_file_date = f.read()
        f.close()
    with open(last_data_datetime_pid_path) as f:
        last_data_datetime = f.read()
        f.close()

    print(zscore_file_date)
    print(encode_file_date)
    print(modal_file_date)
    print(last_data_datetime)

    # 讀取檔案最後日期 [E]
    # 參數設定 [B]
    today_object = datetime.strptime(begin_date, '%Y-%m-%d %H:%M:%S')
    today = today_object.strftime("%Y-%m-%d-%H-%M-%S")
    today_object_string = today_object.strftime("%Y-%m-%d %H:%M:00")
    today_hourly_object = datetime.strptime(today_object.strftime("%Y-%m-%d %H:00:00"), "%Y-%m-%d %H:%M:%S")
    now_quarter_nums = int(today_object.strftime("%M")) // predict_min
    now_quarter_times_object = today_hourly_object + timedelta(minutes=now_quarter_nums*predict_min)
    now_quarter_times = now_quarter_times_object.strftime("%Y-%m-%d %H:%M:00")
    last_data_datetime_object = datetime.strptime(last_data_datetime, "%Y-%m-%d %H:%M:%S")
    begin_date = (last_data_datetime_object - timedelta(minutes=predict_min * window_size)).strftime("%Y-%m-%d %H:%M:%S")
    end_date = today_object_string
    # 參數設定 [E]
    # 讀取 zscore 路徑 [B]
    temperature_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/temperature-zscore-{}.bin'
    pres_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/pres-zscore-{}.bin'
    wind_speed_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/wind_speed-zscore-{}.bin'
    wind_dir_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/wind_dir-zscore-{}.bin'
    max_wind_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/max_wind-zscore-{}.bin'
    r24_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/r24-zscore-{}.bin'
    huvi_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/huvi-zscore-{}.bin'
    apparent_temp_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/apparent_temp-zscore-{}.bin'

    T_Min_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/T_Min-zscore-{}.bin'
    T_Max_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/T_Max-zscore-{}.bin'
    day_of_year_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/day_of_year-zscore-{}.bin'
    data_week_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/data_week-zscore-{}.bin'
    data_hour_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/data_hour-zscore-{}.bin'
    data_minute_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/data_minute-zscore-{}.bin'
    minutes_of_the_day_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/minutes_of_the_day-zscore-{}.bin'
    demand_zscore_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/demand-zscore-{}.bin'

    data_weekday_encode_to_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/zscore/data_weekday-encode-{}.bin'

    simple_LSTM_modal_save_path = '/content/drive/My Drive/ncku_demand/parameter-storage10/modal/simple-LSTM-modal-no-attention.h5'
   
    temperature_zscore_last_path = temperature_zscore_to_path.format(zscore_file_date)
    pres_zscore_last_path = pres_zscore_to_path.format(zscore_file_date)
    wind_speed_zscore_last_path = wind_speed_zscore_to_path.format(zscore_file_date)
    wind_dir_zscore_last_path = wind_dir_zscore_to_path.format(zscore_file_date)
    max_wind_zscore_last_path = max_wind_zscore_to_path.format(zscore_file_date)
    r24_zscore_last_path = r24_zscore_to_path.format(zscore_file_date)
    huvi_zscore_last_path = huvi_zscore_to_path.format(zscore_file_date)
    apparent_temp_zscore_last_path = apparent_temp_zscore_to_path.format(zscore_file_date)

    T_Min_zscore_last_path = T_Min_zscore_to_path.format(zscore_file_date)
    T_Max_zscore_last_path = T_Max_zscore_to_path.format(zscore_file_date)
    day_of_year_zscore_last_path = day_of_year_zscore_to_path.format(zscore_file_date)
    data_week_zscore_last_path = data_week_zscore_to_path.format(zscore_file_date)
    data_hour_zscore_last_path = data_hour_zscore_to_path.format(zscore_file_date)
    data_minute_zscore_last_path = data_minute_zscore_to_path.format(zscore_file_date)
    minutes_of_the_day_zscore_last_path = minutes_of_the_day_zscore_to_path.format(zscore_file_date)
    demand_zscore_last_path = demand_zscore_to_path.format(zscore_file_date)

    data_weekday_encode_last_path = data_weekday_encode_to_path.format(encode_file_date)

    simple_LSTM_modal_save_last_path = simple_LSTM_modal_save_path.format(modal_file_date)
    # 讀取 zscore 路徑 [E]
    # 新的 zscore 路徑 [B]
    temperature_zscore_new_path = temperature_zscore_to_path.format(today)
    pres_zscore_new_path = pres_zscore_to_path.format(today)
    wind_speed_zscore_new_path = wind_speed_zscore_to_path.format(today)
    wind_dir_zscore_new_path = wind_dir_zscore_to_path.format(today)
    max_wind_zscore_new_path = max_wind_zscore_to_path.format(today)
    r24_zscore_new_path = r24_zscore_to_path.format(today)
    huvi_zscore_new_path = huvi_zscore_to_path.format(today)
    apparent_temp_zscore_new_path = apparent_temp_zscore_to_path.format(today)

    T_Min_zscore_new_path = T_Min_zscore_to_path.format(today)
    T_Max_zscore_new_path = T_Max_zscore_to_path.format(today)
    day_of_year_zscore_new_path = day_of_year_zscore_to_path.format(today)
    data_week_zscore_new_path = data_week_zscore_to_path.format(today)
    data_hour_zscore_new_path = data_hour_zscore_to_path.format(today)
    data_minute_zscore_new_path = data_minute_zscore_to_path.format(today)
    minutes_of_the_day_zscore_new_path = minutes_of_the_day_zscore_to_path.format(today)
    demand_zscore_new_path = demand_zscore_to_path.format(today)

    simple_LSTM_modal_save_new_path = simple_LSTM_modal_save_path.format(today)
    # 新的 zscore 路徑 [E]
    # csv 撈取需量資料 [B]
    reframed = dataframe.loc[:end_date, :]
    reframed = reframed.iloc[(window_size * -1)-1:];
    # csv 撈取需量資料 [E]
    # 補缺失值 [B]
    reframed = reframed.interpolate(method="linear") 
    # 補缺失值 [E]
    # 刪除缺失值 [B]
    reframed = reframed.dropna()
    # 刪除缺失值 [E]
    # 排序撈取資料 [B]
    reframed = reframed.sort_index()
    # 排序撈取資料 [E]
    # 體感溫度計算 [B]
    reframed['apparent_temp'] = reframed.apply(lambda reframed: c_feel_like(reframed['Temperature'], reframed['w_DSD']), axis=1)
    # 體感溫度計算 [E]

    reframed = reframed.drop(columns=['id', 'demand_min', 'Total_value', 'data_date'])
    # 轉為監督式資料 [B]
    reframed = series_to_supervised(reframed, 1, 1)
    # 轉為監督式資料 [E]
    # 最後一筆資料截取 [B]
    last_datetime_dataframe = reframed.copy()
    # 最後一筆資料截取 [E]

    # 讀取 zscore 檔案 [B]
    temperature_zscore=load(temperature_zscore_last_path)
    pres_zscore=load(pres_zscore_last_path)
    wind_speed_zscore=load(wind_speed_zscore_last_path)
    wind_dir_zscore=load(wind_dir_zscore_last_path)
    max_wind_zscore=load(max_wind_zscore_last_path)
    r24_zscore=load(r24_zscore_last_path)
    huvi_zscore=load(huvi_zscore_last_path)
    apparent_temp_zscore=load(apparent_temp_zscore_last_path)
    T_Min_zscore = load(T_Min_zscore_last_path)
    T_Max_zscore = load(T_Max_zscore_last_path)
    day_of_year_zscore = load(day_of_year_zscore_last_path)
    data_week_zscore = load(data_week_zscore_last_path) 
    data_hour_zscore = load(data_hour_zscore_last_path)
    data_minute_zscore = load(data_minute_zscore_last_path) 
    minutes_of_the_day_zscore = load(minutes_of_the_day_zscore_last_path)
    demand_zscore = load(demand_zscore_last_path)
    # 讀取 zscore 檔案 [E]
    # 資料 reshape [B]
    temperature_reshape_data = reframed['Temperature(t)'].values.reshape(-1, 1)
    pres_reshape_data = reframed['pres(t)'].values.reshape(-1, 1)
    wind_speed_reshape_data = reframed['w_DSD(t)'].values.reshape(-1, 1)
    wind_dir_reshape_data = reframed['w_DIR(t)'].values.reshape(-1, 1)
    max_wind_reshape_data = reframed['h_FX(t)'].values.reshape(-1, 1)
    r24_reshape_data = reframed['24R(t)'].values.reshape(-1, 1)
    huvi_reshape_data = reframed['h_UVI(t)'].values.reshape(-1, 1)
    apparent_temp_reshape_data = reframed['apparent_temp(t)'].values.reshape(-1, 1)
    T_Min_reshape_data = reframed['T_Min(t)'].values.reshape(-1, 1)
    T_Max_reshape_data = reframed['T_Max(t)'].values.reshape(-1, 1)
    day_of_year_reshape_data = reframed['day_of_year(t)'].values.reshape(-1, 1)
    data_week_reshape_data = reframed['data_week(t)'].values.reshape(-1, 1)
    data_hour_reshape_data = reframed['data_hour(t)'].values.reshape(-1, 1)
    data_minute_reshape_data = reframed['data_minute(t)'].values.reshape(-1, 1)
    minutes_of_the_day_reshape_data = reframed['minutes_of_the_day(t)'].values.reshape(-1, 1)
    last_demand_reshape_data = reframed['demand_quarter(t-1)'].values.reshape(-1, 1)
    demand_reshape_data = reframed['demand_quarter(t)'].values.reshape(-1, 1)
    # 資料 reshape [E]
    # 資料正規化 [B]
    reframed['Temperature(t)'] = temperature_zscore.transform(temperature_reshape_data)
    reframed['pres(t)'] = pres_zscore.transform(pres_reshape_data)
    reframed['w_DSD(t)'] = wind_speed_zscore.transform(wind_speed_reshape_data)
    reframed['w_DIR(t)'] = wind_dir_zscore.transform(wind_dir_reshape_data)
    reframed['h_FX(t)'] = max_wind_zscore.transform(max_wind_reshape_data)
    reframed['24R(t)'] = r24_zscore.transform(r24_reshape_data)
    reframed['h_UVI(t)'] = huvi_zscore.transform(huvi_reshape_data)
    reframed['apparent_temp(t)'] = apparent_temp_zscore.transform(apparent_temp_reshape_data)
    reframed['T_Min(t)'] = T_Min_zscore.transform(T_Min_reshape_data)
    reframed['T_Max(t)'] = T_Max_zscore.transform(T_Max_reshape_data)
    reframed['day_of_year(t)'] = day_of_year_zscore.transform(day_of_year_reshape_data)
    reframed['data_week(t)'] = data_week_zscore.transform(data_week_reshape_data)
    reframed['data_hour(t)'] = data_hour_zscore.transform(data_hour_reshape_data)
    reframed['data_minute(t)'] = data_minute_zscore.transform(data_minute_reshape_data)
    reframed['minutes_of_the_day(t)'] = minutes_of_the_day_zscore.transform(minutes_of_the_day_reshape_data)
    reframed['demand_quarter(t-1)'] = demand_zscore.transform(last_demand_reshape_data)
    reframed['demand_quarter_normalization'] = demand_zscore.transform(demand_reshape_data)
    # 資料正規化 [E]

    # 讀取 one hot encoding 模型 [B]
    data_weekday_encoder = load(data_weekday_encode_last_path)
    # 讀取 one hot encoding 模型 [E]

    # 將分類資料轉為 one hot encoding [B]
    data_weekday_values = reframed['data_weekday(t)'].values.reshape(len(reframed['data_weekday(t)']), 1)
    data_weekday_values = data_weekday_encoder.transform(data_weekday_values)
    data_weekday_encode_dataframe= pd.DataFrame(data_weekday_values)
    for i in data_weekday_encode_dataframe:
        reframed['data_weekday_{}(t)'.format(i)] = data_weekday_encode_dataframe[i].values
    # 將分類資料轉為 one hot encoding [E]

    # 分割訓練資料與測試資料 [B]
    selected_column = ['Temperature(t)', 'humd(t)', 'pres(t)', 'w_DSD(t)', 'w_DIR(t)', 'h_FX(t)', '24R(t)', 'h_UVI(t)',
                   'T_Min(t)', 'T_Max(t)', 'day_of_year(t)', 'data_week(t)', 'data_hour(t)',
                   'data_minute(t)', 'minutes_of_the_day(t)', 'apparent_temp(t)', 'demand_quarter(t-1)', 'data_weekday_0(t)',
                   'data_weekday_1(t)', 'data_weekday_2(t)', 'data_weekday_3(t)', 'data_weekday_4(t)',
                   'data_weekday_5(t)', 'data_weekday_6(t)']
    selected_column_no_weather = ['day_of_year(t)', 'data_week(t)', 'data_hour(t)',
                   'data_minute(t)', 'minutes_of_the_day(t)', 'demand_quarter(t-1)', 'data_weekday_0(t)',
                   'data_weekday_1(t)', 'data_weekday_2(t)', 'data_weekday_3(t)', 'data_weekday_4(t)',
                   'data_weekday_5(t)', 'data_weekday_6(t)']
    all_update_features = np.array([reframed.loc[:, selected_column]])
    update_labels = np.array([reframed.loc[reframed.index[-1], 'demand_quarter_normalization']])
    # 分割訓練資料與測試資料 [E]

    # 讀取模型 [B]
    vanilla_model = create_model(window_size, feature_len, model_structure)
    load_weights(vanilla_model, simple_LSTM_modal_save_last_path)

    # 訓練模型 [B]
    vanilla_model_history = vanilla_model.fit(all_update_features, update_labels)
    vanilla_model.compile(loss='mse', optimizer='Adam') 
    # 訓練模型 [E]
    # 儲存模型 [B]
    #     if (vanilla_model_history.history['loss'][0] > 0.014):
    vanilla_model.save(simple_LSTM_modal_save_last_path) 
    # 儲存模型 [E]
    last_datetime_object = last_datetime_dataframe.iloc[-1]
    last_datetime = '{}'.format(last_datetime_object.T.name)
    predict_features_dataframe = pd.DataFrame({})
    predict_datetime = np.array([])
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    future_features_dataframe = last_datetime_object.to_frame().T.copy()
    last_predict = future_features_dataframe['demand_quarter(t-1)']
    predict_demand = np.array([])
    for i in range(1, 5):
        last_datetime_dataframe = last_datetime_dataframe.iloc[1:].copy()
        # 時間生成 [B]
        future_features_dataframe['demand_quarter(t-1)'] = last_predict
        now_quarter_hour_datetime = now_quarter_times_object.strftime("%Y-%m-%d %H:00:00")
        future_times_object = now_quarter_times_object + timedelta(minutes = i * 15)
        next_hour_object = future_times_object + timedelta(hours = 1)
        future_times = future_times_object.strftime("%Y-%m-%d %H:%M:%S")
        future_minutes =future_times_object.strftime("%M")
        future_hour_datetime = future_times_object.strftime("%Y-%m-%d %H:00:00")
        future_times_api_format = future_times_object.strftime("%Y%m%d")
        future_hour_time = future_times_object.strftime("%H:00")
        next_hour_time = next_hour_object.strftime("%H:00")
        next_hour_datetime = next_hour_object.strftime("%Y-%m-%d %H:00:00")
        next_hour_api_format = next_hour_object.strftime("%Y%m%d")
        # 時間生成 [E]
        # 抓取下一小時天氣 [B]
        if next_hour_datetime < now:
            # 抓資料庫天氣 [B]
            next_hour_weather_table = getWeatherCSV(dataframe, next_hour_datetime)
            next_hour_weather_table['apparent_temp'] = next_hour_weather_table.apply(lambda next_hour_weather_table: c_feel_like(next_hour_weather_table['Temperature'], next_hour_weather_table['w_DSD']), axis=1)
            # 抓資料庫天氣 [E]
        else:
            url = 'https://www.timeanddate.com/weather/@1668355/hourly?hd={}'.format(next_hour_api_format)
            table = pd.read_html(url)
            next_hour_weather_table = table[0].droplevel(level=0, axis=1)
            next_hour_weather_table = next_hour_weather_table[:-1]
            next_hour_weather_table['Time'] = next_hour_weather_table['Time'].str[:5]
            next_hour_weather_table.loc[next_hour_weather_table['Time'] == next_hour_time].reset_index()
        # 抓取下一小時天氣 [E]
        # 抓取鄰近小時天氣 [B]
        if future_hour_datetime < now:
            # 抓資料庫天氣 [B]
            weather_table = getWeatherCSV(dataframe, future_hour_datetime)
            weather_table['apparent_temp'] = weather_table.apply(lambda weather_table: c_feel_like(weather_table['Temperature'], weather_table['w_DSD']), axis=1)
            # 抓資料庫天氣 [E]
        else:
            url = 'https://www.timeanddate.com/weather/@1668355/hourly?hd={}'.format(future_times_api_format)
            table = pd.read_html(url)
            weather_table = table[0].droplevel(level=0, axis=1)
            weather_table = weather_table[:-1]
            weather_table['Time'] = weather_table['Time'].str[:5]
            weather_table.loc[weather_table['Time'] == future_hour_time].reset_index()
        # 抓取鄰近小時天氣 [E]
        # 利用內插法新增溫度 [B]
        old_temp = future_features_dataframe['Temperature(t)'] if future_hour_datetime <= now_quarter_hour_datetime else (weather_table['Temperature'] if next_hour_datetime < now else float(weather_table['Temp'][0].replace(" °C", "")))
        old_humd = future_features_dataframe['humd(t)'] if future_hour_datetime <= now_quarter_hour_datetime else (weather_table['humd'] if next_hour_datetime < now else float(weather_table['Humidity'][0].replace(" °C", "")))
        old_wdsd = future_features_dataframe['w_DSD(t)'] if future_hour_datetime <= now_quarter_hour_datetime else (weather_table['w_DSD'] if next_hour_datetime < now else float(weather_table['Wind'][0].replace(" °C", "")))
        old_apptemp = future_features_dataframe['apparent_temp(t)'] if future_hour_datetime <= now_quarter_hour_datetime else (weather_table['apparent_temp'] if next_hour_datetime < now else float(weather_table['Feels Like'][0].replace(" °C", "")))
        hour_temp = next_hour_weather_table['Temperature'] if next_hour_datetime < now else float(next_hour_weather_table['Temp'][0].replace(" °C", ""))
        hour_humd = next_hour_weather_table['humd'] if next_hour_datetime < now else float(next_hour_weather_table['Humidity'][0].replace("%", ""))
        hour_wdsd = next_hour_weather_table['w_DSD'] if next_hour_datetime < now else float(next_hour_weather_table['Wind'][0].replace(" km/h", ""))
        hour_apptemp = next_hour_weather_table['apparent_temp'] if next_hour_datetime < now else float(next_hour_weather_table['Feels Like'][0].replace(" °C", ""))
        hour_temp = float(old_temp) + ((float(hour_temp) - float(old_temp)) * (float(future_minutes)/60))
        hour_humd = float(old_humd) + ((float(hour_humd) - float(old_humd)) * (float(future_minutes)/60))
        hour_wdsd = float(old_wdsd) + ((float(hour_wdsd) - float(old_wdsd)) * (float(future_minutes)/60))
        hour_apptemp = float(old_apptemp) + ((float(hour_apptemp) - float(old_apptemp)) * (float(future_minutes)/60))
        # 利用內插法新增溫度 [E]
        # 預測陣列 [B]
        predict_datetime = np.append(predict_datetime, future_times)
        future_features_dataframe['day_of_year(t)'] = int(future_times_object.strftime("%j"))
        future_features_dataframe['data_week(t)'] = int(future_times_object.strftime("%W"))
        future_features_dataframe['data_hour(t)'] = int(future_times_object.strftime("%H"))
        future_features_dataframe['data_minute(t)'] = int(future_times_object.strftime("%M"))
        future_features_dataframe['data_weekday(t)'] = int(future_times_object.strftime("%w")) + 1
        future_minutes_of_the_day = future_features_dataframe['data_hour(t)'] * 60 + future_features_dataframe['data_minute(t)']
        future_features_dataframe['Temperature(t)'] = hour_temp
        future_features_dataframe['humd(t)'] = hour_humd
        future_features_dataframe['w_DSD(t)'] = hour_wdsd
        future_features_dataframe['apparent_temp(t)'] = hour_apptemp
        last_datetime_dataframe = last_datetime_dataframe.append(future_features_dataframe)
        normalization_future_dataframe = last_datetime_dataframe.copy()
        # 預測陣列 [B]
        # 資料 reshape [B]
        new_temperature_reshape_data = normalization_future_dataframe['Temperature(t)'].values.reshape(-1, 1)
        new_pres_reshape_data = normalization_future_dataframe['pres(t)'].values.reshape(-1, 1)
        new_wind_speed_reshape_data = normalization_future_dataframe['w_DSD(t)'].values.reshape(-1, 1)
        new_wind_dir_reshape_data = normalization_future_dataframe['w_DIR(t)'].values.reshape(-1, 1)
        new_max_wind_reshape_data = normalization_future_dataframe['h_FX(t)'].values.reshape(-1, 1)
        new_r24_reshape_data = normalization_future_dataframe['24R(t)'].values.reshape(-1, 1)
        new_huvi_reshape_data = normalization_future_dataframe['h_UVI(t)'].values.reshape(-1, 1)
        new_apparent_temp_data = normalization_future_dataframe['apparent_temp(t)'].values.reshape(-1, 1)
        new_T_Min_reshape_data = normalization_future_dataframe['T_Min(t)'].values.reshape(-1, 1)
        new_T_Max_reshape_data = normalization_future_dataframe['T_Max(t)'].values.reshape(-1, 1)
        new_day_of_year_reshape_data = normalization_future_dataframe['day_of_year(t)'].values.reshape(-1, 1)
        new_data_week_reshape_data = normalization_future_dataframe['data_week(t)'].values.reshape(-1, 1)
        new_data_hour_reshape_data = normalization_future_dataframe['data_hour(t)'].values.reshape(-1, 1)
        new_data_minute_reshape_data = normalization_future_dataframe['data_minute(t)'].values.reshape(-1, 1)
        new_minutes_of_the_day_reshape_data = normalization_future_dataframe['minutes_of_the_day(t)'].values.reshape(-1, 1)
        new_last_demand_reshape_data = normalization_future_dataframe['demand_quarter(t-1)'].values.reshape(-1, 1)
        # 資料 reshape [E]
        # 資料正規化 [B]
        normalization_future_dataframe['Temperature(t)'] = temperature_zscore.transform(new_temperature_reshape_data)
        normalization_future_dataframe['pres(t)'] = pres_zscore.transform(new_pres_reshape_data)
        normalization_future_dataframe['w_DSD(t)'] = wind_speed_zscore.transform(new_wind_speed_reshape_data)
        normalization_future_dataframe['w_DIR(t)'] = wind_dir_zscore.transform(new_wind_dir_reshape_data)
        normalization_future_dataframe['h_FX(t)'] = max_wind_zscore.transform(new_max_wind_reshape_data)
        normalization_future_dataframe['24R(t)'] = r24_zscore.transform(new_r24_reshape_data)
        normalization_future_dataframe['h_UVI(t)'] = huvi_zscore.transform(new_huvi_reshape_data)
        normalization_future_dataframe['apparent_temp(t)'] = apparent_temp_zscore.transform(new_apparent_temp_data)
        normalization_future_dataframe['T_Min(t)'] = T_Min_zscore.transform(new_T_Min_reshape_data)
        normalization_future_dataframe['T_Max(t)'] = T_Max_zscore.transform(new_T_Max_reshape_data)
        normalization_future_dataframe['day_of_year(t)'] = day_of_year_zscore.transform(new_day_of_year_reshape_data)
        normalization_future_dataframe['data_week(t)'] = data_week_zscore.transform(new_data_week_reshape_data)
        normalization_future_dataframe['data_hour(t)'] = data_hour_zscore.transform(new_data_hour_reshape_data)
        normalization_future_dataframe['data_minute(t)'] = data_minute_zscore.transform(new_data_minute_reshape_data)
        normalization_future_dataframe['minutes_of_the_day(t)'] = minutes_of_the_day_zscore.transform(new_minutes_of_the_day_reshape_data)
        normalization_future_dataframe['demand_quarter(t-1)'] = demand_zscore.transform(new_last_demand_reshape_data)
        # 資料正規化 [E]
        # 將分類資料轉為 one hot encoding [B]
        predict_data_weekday_values = normalization_future_dataframe['data_weekday(t)'].values.reshape(len(normalization_future_dataframe['data_weekday(t)']), 1)
        predict_data_weekday_values = data_weekday_encoder.transform(predict_data_weekday_values)

        predict_data_weekday_encode_dataframe= pd.DataFrame(predict_data_weekday_values)

        for i in predict_data_weekday_encode_dataframe:
            normalization_future_dataframe['data_weekday_{}(t)'.format(i)] = predict_data_weekday_encode_dataframe[i].values
        normalization_future_dataframe =  normalization_future_dataframe.drop(columns=['data_weekday(t)'])
        # 將分類資料轉為 one hot encoding [E]
        normalization_future_dataframe = np.array([normalization_future_dataframe.loc[:, selected_column]])

        # 預測 [B]
        vanilla_predict_demands = vanilla_model.predict(normalization_future_dataframe)
        # 預測 [E]
        # 一般預測 [E]
        # 還原 z-score [B]
        vanilla_predict_demands = demand_zscore.inverse_transform(vanilla_predict_demands)
        last_predict = vanilla_predict_demands[0][0]
        predict_demand = np.append(predict_demand, vanilla_predict_demands)

    # 預測上傳 [B]
    upload_table_name = ['vanilla_demand_predict_no_attention']
    upload_dataframe = [predict_demand]
    i = 0
    for predict_demands in upload_dataframe:
        uploadPredictionCSV(upload_table_name[i], now_quarter_times, predict_datetime, predict_demands)
        i = i + 1
    # 預測上傳 [E]
    # 儲存撈取最後一筆的時間 [B]
    last_data_datetime_pid_write = open(last_data_datetime_pid_path, 'w', encoding='utf-8')
    last_data_datetime_pid_write.write(last_datetime)
    last_data_datetime_pid_write.close()
    # 儲存撈取最後一筆的時間 [E]
    
    # 刪除模型 [B]
    del vanilla_model
    # 刪除模型 [E]
''' 需量預測函式，在 codlab 上透過撈取 csv 資訊來做預測 [E]'''

In [ ]:
# 要求 codlab 連線至我的 google drive [B]
from google.colab import drive
drive.mount('/content/drive')
# 要求 codlab 連線至我的 google drive [E]

Mounted at /content/drive


In [ ]:
# 先撈取資料 [B]
reframed = pd.read_csv("/content/drive/My Drive/ncku_demand/demand_with_weather_data-20201229-20220102.csv")

In [ ]:
timestamp = pd.to_datetime(reframed.datetime, infer_datetime_format=True).values.astype(float)
reframed['timestamp'] = timestamp.tolist()
reframed = reframed.set_index('datetime')
# 先撈取資料 [E]

In [ ]:
datelist = pd.date_range(start='2021-12-25 19:16:00', end='2022-01-01', freq='15min')

In [ ]:
for date in datelist:
      predict_demand_csv(reframed, '{}'.format(date), 'ncku_demand', 'no-attention')

2022-07-29-13-31-28
2022-07-29-13-31-28
2022-07-29-10-53-04
2021-12-25 19:00:00


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


1/1 [==============================] - 6s 6s/step - loss: 1.5316e-05
2022-07-29-13-31-28
2022-07-29-13-31-28
2022-07-29-10-53-04
2021-12-25 19:15:00


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


1/1 [==============================] - 1s 1s/step - loss: 5.8880e-05
2022-07-29-13-31-28
2022-07-29-13-31-28
2022-07-29-10-53-04
2021-12-25 19:30:00


...

1/1 [==============================] - 1s 1s/step - loss: 8.6624e-05
2022-07-29-13-31-28
2022-07-29-13-31-28
2022-07-29-10-53-04
2021-12-31 23:15:00


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


1/1 [==============================] - 1s 1s/step - loss: 1.1037e-04
2022-07-29-13-31-28
2022-07-29-13-31-28
2022-07-29-10-53-04
2021-12-31 23:30:00


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


1/1 [==============================] - 1s 1s/step - loss: 1.6684e-06
